# **APHIS Dataset Cleaning File**

This file includes code to clean the APHIS dataset, which includes our target variable (yield per colony), as well as various feature variables, including levels of spores and indicators on the presence of different bee viruses.


**Import packages**

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

pd.set_option('display.max_rows', 20)


**Mount the drive**


In [2]:
# from google.colab import drive
# drive.mount("/content/drive", force_remount=True)

**Read in the CSV file**

In [3]:
df_aphis = pd.read_csv("../04 - Data/Raw_Data/aphis_state_year_data.a7201cb783e0.csv", skiprows=range(9))
df_aphis_col = pd.read_csv("../04 - Data/Raw_Data/aphis_state_year_data.a7201cb783e0.csv", skiprows = lambda x: x not in [8,9]).columns
df_aphis.columns = df_aphis_col

# How many rows in the dataset? 
len(df_aphis) # 8298

# Display header for good measure: 
df_aphis.head()
#df_aphis.sort_values(by = ["state_code", "sample_year"])


,sample_year,sample_month_number,sample_month,state_code,sampling_county,varroa_per_100_bees,million_spores_per_bee,abpv,abpv_percentile,cbpv,...,iapv_percentile,kbv,kbv_percentile,lsv2,lsv2_percentile,sbpv,sbpv_percentile,mkv,mkv_percentile,pesticides
0,2009,7,July,HI,NaN,0.00,0.02,NaN,NaN,-,...,NaN,-,NaN,NaN,NaN,-,NaN,NaN,NaN,NaN
1,2009,7,July,HI,NaN,3.46,0.03,NaN,NaN,-,...,NaN,-,NaN,NaN,NaN,-,NaN,NaN,NaN,NaN
2,2009,7,July,HI,NaN,11.48,0.03,NaN,NaN,-,...,NaN,-,NaN,NaN,NaN,-,NaN,NaN,NaN,NaN
3,2009,7,July,HI,NaN,14.00,0.08,NaN,NaN,-,...,NaN,+,NaN,NaN,NaN,-,NaN,NaN,NaN,NaN
4,2009,7,July,HI,NaN,1.15,0.03,NaN,NaN,-,...,NaN,-,NaN,NaN,NaN,-,NaN,NaN,NaN,NaN


**Create a function to transform the + and - sign indicators into binary {0,1} data**

In [4]:
def transfBinary(value):
    if value == "+": # if positive, make a 1
        return 1
    elif value == "-": # if negative, make a 0; leaves NaN
        return 0
  

**Apply the function to the dataset**




In [5]:
pathogens = ['cbpv', 'dwv', 'iapv', 'kbv', 'sbpv', 'abpv', 'dwv-b', 'lsv2', 'mkv']

aphis = pd.DataFrame()
aphis[["year", "month", "state", "varroa", "spores"]] = df_aphis[["sample_year", "sample_month_number", "state_code", "varroa_per_100_bees", "million_spores_per_bee"]]


for i in pathogens:
     aphis[i] = df_aphis[i].apply(transfBinary)


aphis
#aphis["cbpv"].sort_values(ascending= False)

,year,month,state,varroa,spores,cbpv,dwv,iapv,kbv,sbpv,abpv,dwv-b,lsv2,mkv
0,2009,7,HI,0.00,0.02,0.0,1.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN
1,2009,7,HI,3.46,0.03,0.0,1.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN
2,2009,7,HI,11.48,0.03,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN
3,2009,7,HI,14.00,0.08,0.0,1.0,0.0,1.0,0.0,NaN,NaN,NaN,NaN
4,2009,7,HI,1.15,0.03,0.0,1.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8293,2020,12,FL,0.38,0.30,1.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
8294,2020,12,FL,9.74,0.35,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
8295,2020,12,FL,0.00,0.60,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0
8296,2020,12,FL,0.14,0.30,0.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0


 **Impute Missing Values**

In [6]:
# distinguish the continues and binary variables
pathogen_cont = ['varroa', 'spores']
pathogen_binary = ['cbpv', 'dwv', 'iapv', 'kbv', 'sbpv', 'abpv', 'dwv-b', 'lsv2', 'mkv']

for i in pathogen_cont:
  aphis[i] = aphis.groupby(["year","state"])[i].transform(lambda x: x.fillna(x.mean()))
  aphis[i] = aphis.groupby("state")[i].transform(lambda x: x.fillna(x.mean()))
  aphis[i] = aphis.groupby("year")[i].transform(lambda x: x.fillna(x.mean()))

for i in pathogen_binary:
  aphis[i] = aphis.groupby(["year","state"])[i].transform(lambda x: x.fillna(x.mode()[0] if len(x.mode()) == 1 else x))
  aphis[i] = aphis.groupby("state")[i].transform(lambda x: x.fillna(x.mode()[0] if len(x.mode()) == 1 else x))
  aphis[i] = aphis.groupby("year")[i].transform(lambda x: x.fillna(x.mode()[0] if len(x.mode()) == 1 else x))

aphis

,year,month,state,varroa,spores,cbpv,dwv,iapv,kbv,sbpv,abpv,dwv-b,lsv2,mkv
0,2009,7,HI,0.00,0.02,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,2009,7,HI,3.46,0.03,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,2009,7,HI,11.48,0.03,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3,2009,7,HI,14.00,0.08,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
4,2009,7,HI,1.15,0.03,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8293,2020,12,FL,0.38,0.30,1.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
8294,2020,12,FL,9.74,0.35,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
8295,2020,12,FL,0.00,0.60,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0
8296,2020,12,FL,0.14,0.30,0.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0


**Group by state, year and calculate mean for the continuous variables and max value and probability for binary variables**

In [7]:
# distinguish the continues and binary variables
pathogen_cont = ['varroa', 'spores']
pathogen_binary = ['cbpv', 'dwv', 'iapv', 'kbv', 'sbpv', 'abpv', 'dwv-b', 'lsv2', 'mkv']

bee_cont = pd.DataFrame()
bee_max = pd.DataFrame()
bee_prob = pd.DataFrame()

for i in pathogen_cont:
  bee_cont[i] = aphis.groupby(['year', 'state'])[i].mean()
bee_cont = bee_cont.reset_index()

for i in pathogen_binary:
  bee_max[i] = aphis.groupby(['year', 'state'])[i].max()
bee_max = bee_max.reset_index()

bee_max = bee_max.rename({'cbpv':'cbpv_max', 'dwv':'dwv_max', 'iapv':'iapv_max', 
                            'kbv':'kbv_max', 'sbpv':'sbpv_max', 'abpv':'abpv_max', 
                            'dwv-b':'dwv-b_max', 'lsv2':'lsv2_max', 'mkv':'mkv_max'}, axis=1)

for i in pathogen_binary:
  bee_prob[i] = aphis.groupby(['year', 'state'])[i].mean()
bee_prob = bee_prob.reset_index()

bee_prob = bee_prob.rename({'cbpv':'cbpv_prob', 'dwv':'dwv_prob', 'iapv':'iapv_prob', 
                            'kbv':'kbv_prob', 'sbpv':'sbpv_prob', 'abpv':'abpv_prob', 
                            'dwv-b':'dwv-b_prob', 'lsv2':'lsv2_prob', 'mkv':'mkv_prob'}, axis=1)

#'''
df_aphis2 = bee_cont.join(bee_max.set_index(['year', 'state']), on=['year', 'state'])
df_aphis2 = df_aphis2.join(bee_prob.set_index(['year', 'state']), on=['year', 'state'])
df_aphis2
#'''

,year,state,varroa,spores,cbpv_max,dwv_max,iapv_max,kbv_max,sbpv_max,abpv_max,...,mkv_max,cbpv_prob,dwv_prob,iapv_prob,kbv_prob,sbpv_prob,abpv_prob,dwv-b_prob,lsv2_prob,mkv_prob
0,2009,CA,1.830833,0.175000,0.0,1.0,1.0,1.0,0.0,0.0,...,0.0,0.000000,0.666667,0.166667,0.166667,0.0,0.000000,1.000000,0.000000,0.0
1,2009,HI,7.344286,0.645714,0.0,1.0,0.0,1.0,0.0,1.0,...,0.0,0.000000,0.714286,0.000000,0.142857,0.0,1.000000,0.000000,0.000000,0.0
2,2010,AL,3.390800,0.079600,0.0,1.0,0.0,1.0,0.0,1.0,...,0.0,0.000000,1.000000,0.000000,0.040000,0.0,0.040000,1.000000,0.000000,0.0
3,2010,CA,1.282424,0.759394,1.0,1.0,1.0,1.0,0.0,0.0,...,0.0,0.028571,0.742857,0.371429,0.285714,0.0,0.000000,1.000000,0.000000,0.0
4,2010,FL,2.568333,0.745833,1.0,1.0,1.0,1.0,0.0,0.0,...,0.0,0.041667,0.916667,0.166667,0.041667,0.0,0.000000,1.000000,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
373,2020,UT,3.264000,0.143333,1.0,1.0,1.0,0.0,0.0,1.0,...,0.0,0.066667,0.600000,0.266667,0.000000,0.0,0.066667,0.933333,0.466667,0.0
374,2020,VA,3.547917,0.068750,1.0,1.0,0.0,0.0,0.0,1.0,...,0.0,0.083333,0.833333,0.000000,0.000000,0.0,0.083333,0.958333,0.041667,0.0
375,2020,VT,3.736667,0.158333,1.0,1.0,1.0,0.0,0.0,1.0,...,0.0,0.083333,0.750000,0.125000,0.000000,0.0,0.416667,1.000000,0.083333,0.0
376,2020,WI,2.781739,0.045652,1.0,1.0,1.0,0.0,0.0,1.0,...,0.0,0.086957,0.826087,0.478261,0.000000,0.0,0.304348,1.000000,0.391304,0.0


**Create previous years' features**

In [8]:
prev = df_aphis2[['year', 'state', 'varroa', 'spores']]
prev['NextYear'] = prev['year'] + 1
prev.rename(columns={'varroa':'prevyearvarroa', 'spores':'prevyearspores'}, inplace=True)
prev

C:\Users\smail\AppData\Local\Temp\ipykernel_6176\951277231.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  prev['NextYear'] = prev['year'] + 1
C:\Users\smail\anaconda3\lib\site-packages\pandas\core\frame.py:4441: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


,year,state,prevyearvarroa,prevyearspores,NextYear
0,2009,CA,1.830833,0.175000,2010
1,2009,HI,7.344286,0.645714,2010
2,2010,AL,3.390800,0.079600,2011
3,2010,CA,1.282424,0.759394,2011
4,2010,FL,2.568333,0.745833,2011
...,...,...,...,...,...
373,2020,UT,3.264000,0.143333,2021
374,2020,VA,3.547917,0.068750,2021
375,2020,VT,3.736667,0.158333,2021
376,2020,WI,2.781739,0.045652,2021


In [9]:
# Join previous year's yield
interim_df = df_aphis2.merge(prev[['NextYear', 'prevyearspores', 'prevyearvarroa', 'state']], left_on=['year', 'state'], right_on=['NextYear', 'state'], how='left')
interim_df.drop(columns='NextYear', inplace=True)
df = interim_df.copy()
df

,year,state,varroa,spores,cbpv_max,dwv_max,iapv_max,kbv_max,sbpv_max,abpv_max,...,dwv_prob,iapv_prob,kbv_prob,sbpv_prob,abpv_prob,dwv-b_prob,lsv2_prob,mkv_prob,prevyearspores,prevyearvarroa
0,2009,CA,1.830833,0.175000,0.0,1.0,1.0,1.0,0.0,0.0,...,0.666667,0.166667,0.166667,0.0,0.000000,1.000000,0.000000,0.0,NaN,NaN
1,2009,HI,7.344286,0.645714,0.0,1.0,0.0,1.0,0.0,1.0,...,0.714286,0.000000,0.142857,0.0,1.000000,0.000000,0.000000,0.0,NaN,NaN
2,2010,AL,3.390800,0.079600,0.0,1.0,0.0,1.0,0.0,1.0,...,1.000000,0.000000,0.040000,0.0,0.040000,1.000000,0.000000,0.0,NaN,NaN
3,2010,CA,1.282424,0.759394,1.0,1.0,1.0,1.0,0.0,0.0,...,0.742857,0.371429,0.285714,0.0,0.000000,1.000000,0.000000,0.0,0.175000,1.830833
4,2010,FL,2.568333,0.745833,1.0,1.0,1.0,1.0,0.0,0.0,...,0.916667,0.166667,0.041667,0.0,0.000000,1.000000,0.000000,0.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
373,2020,UT,3.264000,0.143333,1.0,1.0,1.0,0.0,0.0,1.0,...,0.600000,0.266667,0.000000,0.0,0.066667,0.933333,0.466667,0.0,0.080000,1.736333
374,2020,VA,3.547917,0.068750,1.0,1.0,0.0,0.0,0.0,1.0,...,0.833333,0.000000,0.000000,0.0,0.083333,0.958333,0.041667,0.0,0.306667,4.964333
375,2020,VT,3.736667,0.158333,1.0,1.0,1.0,0.0,0.0,1.0,...,0.750000,0.125000,0.000000,0.0,0.416667,1.000000,0.083333,0.0,0.168182,2.375909
376,2020,WI,2.781739,0.045652,1.0,1.0,1.0,0.0,0.0,1.0,...,0.826087,0.478261,0.000000,0.0,0.304348,1.000000,0.391304,0.0,0.439583,1.504167


In [10]:
# Fill prevyearvarroa with state average
state_avg_varr = df[['state', 'varroa']]
state_avg_varr = state_avg_varr.groupby('state').mean()
state_avg_varr.reset_index(inplace=True)
state_avg_varr.rename(columns={'varroa':'avgvarroa'}, inplace=True)

interim_df = df.copy()
interim_df = interim_df.merge(right=state_avg_varr, on='state', how='left')
interim_df.prevyearvarroa = np.where(interim_df.prevyearvarroa.isnull(), interim_df.avgvarroa, interim_df.prevyearvarroa)
interim_df.drop(columns='avgvarroa', inplace=True)
df = interim_df.copy()

# Fill prevyearspores with state average
state_avg_spores = df[['state', 'spores']]
state_avg_spores = state_avg_spores.groupby('state').mean()
state_avg_spores.reset_index(inplace=True)
state_avg_spores.rename(columns={'spores':'avgspores'}, inplace=True)

interim_df2 = df.copy()
interim_df2 = interim_df2.merge(right=state_avg_spores, on='state', how='left')
interim_df2.prevyearspores = np.where(interim_df2.prevyearspores.isnull(), interim_df2.avgspores, interim_df2.prevyearspores)
interim_df2.drop(columns='avgspores', inplace=True)
df = interim_df2.copy()

df

,year,state,varroa,spores,cbpv_max,dwv_max,iapv_max,kbv_max,sbpv_max,abpv_max,...,dwv_prob,iapv_prob,kbv_prob,sbpv_prob,abpv_prob,dwv-b_prob,lsv2_prob,mkv_prob,prevyearspores,prevyearvarroa
0,2009,CA,1.830833,0.175000,0.0,1.0,1.0,1.0,0.0,0.0,...,0.666667,0.166667,0.166667,0.0,0.000000,1.000000,0.000000,0.0,0.640246,1.506657
1,2009,HI,7.344286,0.645714,0.0,1.0,0.0,1.0,0.0,1.0,...,0.714286,0.000000,0.142857,0.0,1.000000,0.000000,0.000000,0.0,0.941144,2.632097
2,2010,AL,3.390800,0.079600,0.0,1.0,0.0,1.0,0.0,1.0,...,1.000000,0.000000,0.040000,0.0,0.040000,1.000000,0.000000,0.0,0.124277,5.153710
3,2010,CA,1.282424,0.759394,1.0,1.0,1.0,1.0,0.0,0.0,...,0.742857,0.371429,0.285714,0.0,0.000000,1.000000,0.000000,0.0,0.175000,1.830833
4,2010,FL,2.568333,0.745833,1.0,1.0,1.0,1.0,0.0,0.0,...,0.916667,0.166667,0.041667,0.0,0.000000,1.000000,0.000000,0.0,0.550878,3.866825
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
373,2020,UT,3.264000,0.143333,1.0,1.0,1.0,0.0,0.0,1.0,...,0.600000,0.266667,0.000000,0.0,0.066667,0.933333,0.466667,0.0,0.080000,1.736333
374,2020,VA,3.547917,0.068750,1.0,1.0,0.0,0.0,0.0,1.0,...,0.833333,0.000000,0.000000,0.0,0.083333,0.958333,0.041667,0.0,0.306667,4.964333
375,2020,VT,3.736667,0.158333,1.0,1.0,1.0,0.0,0.0,1.0,...,0.750000,0.125000,0.000000,0.0,0.416667,1.000000,0.083333,0.0,0.168182,2.375909
376,2020,WI,2.781739,0.045652,1.0,1.0,1.0,0.0,0.0,1.0,...,0.826087,0.478261,0.000000,0.0,0.304348,1.000000,0.391304,0.0,0.439583,1.504167


In [11]:
df = df.rename(columns={'state':'code'}) #to allow for matching state name
df = df.sort_values(by = ["code", "year"])
df

,year,code,varroa,spores,cbpv_max,dwv_max,iapv_max,kbv_max,sbpv_max,abpv_max,...,dwv_prob,iapv_prob,kbv_prob,sbpv_prob,abpv_prob,dwv-b_prob,lsv2_prob,mkv_prob,prevyearspores,prevyearvarroa
265,2018,AK,1.176000,0.700000,1.0,1.0,0.0,0.0,0.0,0.0,...,0.300000,0.000000,0.000000,0.0,0.000000,1.000000,0.400000,0.1,0.510000,0.915000
308,2019,AK,0.654000,0.320000,0.0,1.0,1.0,0.0,0.0,0.0,...,0.400000,0.200000,0.000000,0.0,0.000000,1.000000,0.600000,0.0,0.700000,1.176000
2,2010,AL,3.390800,0.079600,0.0,1.0,0.0,1.0,0.0,1.0,...,1.000000,0.000000,0.040000,0.0,0.040000,1.000000,0.000000,0.0,0.124277,5.153710
15,2011,AL,7.364000,0.038000,0.0,1.0,1.0,0.0,0.0,0.0,...,1.000000,0.040000,0.000000,0.0,0.000000,1.000000,0.000000,0.0,0.079600,3.390800
48,2012,AL,7.295833,0.166667,1.0,1.0,0.0,1.0,0.0,1.0,...,1.000000,0.000000,0.250000,0.0,0.041667,1.000000,0.000000,0.0,0.038000,7.364000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
221,2016,WV,4.315833,0.043750,1.0,1.0,0.0,0.0,0.0,1.0,...,0.916667,0.000000,0.000000,0.0,0.291667,0.416667,0.166667,0.0,0.018750,4.557917
264,2017,WV,5.093750,0.216667,1.0,1.0,1.0,1.0,0.0,1.0,...,0.875000,0.083333,0.041667,0.0,0.125000,0.583333,0.291667,0.0,0.043750,4.315833
307,2018,WV,1.947500,0.116667,0.0,1.0,1.0,0.0,0.0,1.0,...,0.666667,0.083333,0.000000,0.0,0.041667,0.500000,0.333333,0.0,0.216667,5.093750
344,2019,WV,4.464074,0.300000,1.0,1.0,1.0,0.0,0.0,1.0,...,0.740741,0.111111,0.000000,0.0,0.037037,0.851852,0.259259,0.0,0.116667,1.947500


In [12]:
state_abbrv = pd.read_csv("../04 - Data/Raw_Data/state_abbrv.csv")
state_abbrv.head()

,state,abbrev,code
0,Alabama,Ala.,AL
1,Alaska,Alaska,AK
2,Arizona,Ariz.,AZ
3,Arkansas,Ark.,AR
4,California,Calif.,CA


In [13]:
df2 = df.join(state_abbrv.set_index("code"), on="code").drop(columns=["abbrev"])
df2.head()

,year,code,varroa,spores,cbpv_max,dwv_max,iapv_max,kbv_max,sbpv_max,abpv_max,...,iapv_prob,kbv_prob,sbpv_prob,abpv_prob,dwv-b_prob,lsv2_prob,mkv_prob,prevyearspores,prevyearvarroa,state
265,2018,AK,1.176000,0.700000,1.0,1.0,0.0,0.0,0.0,0.0,...,0.00,0.00,0.0,0.000000,1.0,0.4,0.1,0.510000,0.91500,Alaska
308,2019,AK,0.654000,0.320000,0.0,1.0,1.0,0.0,0.0,0.0,...,0.20,0.00,0.0,0.000000,1.0,0.6,0.0,0.700000,1.17600,Alaska
2,2010,AL,3.390800,0.079600,0.0,1.0,0.0,1.0,0.0,1.0,...,0.00,0.04,0.0,0.040000,1.0,0.0,0.0,0.124277,5.15371,Alabama
15,2011,AL,7.364000,0.038000,0.0,1.0,1.0,0.0,0.0,0.0,...,0.04,0.00,0.0,0.000000,1.0,0.0,0.0,0.079600,3.39080,Alabama
48,2012,AL,7.295833,0.166667,1.0,1.0,0.0,1.0,0.0,1.0,...,0.00,0.25,0.0,0.041667,1.0,0.0,0.0,0.038000,7.36400,Alabama


In [14]:
df2.to_csv('../04 - Data/Final Data/aphis_clean.csv', index = False)